In [26]:
import statsapi
import pandas as pd
from pandas.io.json import json_normalize

# Get one game from API
test_game = statsapi.get('game_playByPlay',{'gamePk':565997})

# Only care about the allPlays key 
all_plays = test_game.get('allPlays')

# Coerce all plays into a df
test_df = json_normalize(all_plays)

###################################
# Build target table
###################################
# Append dicts to this list
list_for_new_df = []

# Now go through each row. If there is nested list, json_normalize it
for index, row in test_df.head(2).iterrows(): #Just using first 2 rows for testing
    
    # saw playEvents is a nested list, so json_normalize it
    play_events_df = json_normalize(row['playEvents'])
        
    # Loop through THIS NESTED dataframe and NOW build the row for the new df    
    for pev_ind, pev_row in play_events_df.iterrows():
  
        # Instantiate new dict, which will be a single row in target df
        curr_dict = {}
    
        # atBatIndex
        curr_dict['at_bat_index'] = row['about.atBatIndex']

        # batside
        curr_dict['batside'] = row['matchup.batSide.code']

        # pitchHand
        curr_dict['pitchhand'] = row['matchup.pitchHand.code']
        
        # Data from playEvents
        curr_dict['pitch_label'] = pev_row['details.type.description']
        curr_dict['pitch_num'] = pev_row['pitchNumber']
        curr_dict['speed_start'] = pev_row['pitchData.startSpeed']
        curr_dict['speed_end'] = pev_row['pitchData.endSpeed']
        
        # collect row dictionary into list
        list_for_new_df.append(curr_dict)

# Proof of concept on target dataframe
new_df = pd.DataFrame(list_for_new_df)

new_df.head(10)

,at_bat_index,batside,pitch_label,pitch_num,pitchhand,speed_end,speed_start
0,0,R,Four-Seam Fastball,1,L,76.4,84.3
1,0,R,Two-Seam Fastball,2,L,76.3,84.2
2,0,R,Two-Seam Fastball,3,L,77.0,84.9
3,0,R,Changeup,4,L,73.4,80.8
4,0,R,Two-Seam Fastball,5,L,78.0,85.2
5,0,R,Four-Seam Fastball,6,L,79.1,86.9
6,1,R,Four-Seam Fastball,1,L,77.9,85.4
7,1,R,Four-Seam Fastball,2,L,78.0,86.0
8,1,R,Two-Seam Fastball,3,L,77.3,85.5
9,1,R,Four-Seam Fastball,4,L,78.8,87.0
